In [2]:
from tfob import TFOb, BHSA, DSS

**Locating corpus resources ...**

Name,# of nodes,# slots/node,% coverage
scroll,1001,1428.81,100
lex,10450,129.14,94
fragment,11182,127.91,100
line,52895,27.04,100
clause,125,12.85,0
cluster,101099,6.68,47
phrase,315,5.10,0
word,500995,2.81,99
sign,1430241,1.00,100


**Locating corpus resources ...**

Name,# of nodes,# slots/node,% coverage
book,39,10938.21,100
chapter,929,459.19,100
lex,9230,46.22,100
verse,23213,18.38,100
half_verse,45179,9.44,100
sentence,63717,6.70,100
sentence_atom,64514,6.61,100
clause,88131,4.84,100
clause_atom,90704,4.70,100
phrase,253203,1.68,100


In [3]:
from Bio import pairwise2
from Bio.Seq import Seq
import pandas as pd
from itertools import chain

In [4]:
# Function to align verses (Martijn Naaijer)

def align_verse(str_1, str_2):
    
    seq1 = Seq(str_1)
    seq2 = Seq(str_2)
    
    alignements = pairwise2.align.globalxx(seq1, seq2)
    
    seq1_al = (alignements[0][0].strip(' '))
    seq2_al = (alignements[0][1].strip(' '))
               
    return seq1_al, seq2_al

### 1.Create two lists with the verses from MT Isaiah and 1QIsaa

#### 1.1 List of wanted motion verbs

In [5]:
motion_verbs = ["BW>[", "HLK[", "CWB[", "<LH[", "CLX[", "JY>[", 
                "JRD[", "<BR[", "NGC[", "QRB[", "NWS[", "CLK[", 
                ">SP[", "NPL[", "QBY[", "LQX[", "SWR[", "GLH[",
                "QHL["]

#### 1.2 List of occurrences from MT Isaiah containing a motion verbs

In [6]:
verbs_bhsa = TFOb.all("word", BHSA).filter(book="Isaiah").filter_in(lex=motion_verbs)

In [23]:
#verbs_bhsa

In [8]:
TFOb(215297, BHSA).to_verses.text

'דָּבָ֛ר שָׁלַ֥ח אֲדֹנָ֖י בְּיַעֲקֹ֑ב וְנָפַ֖ל בְּיִשְׂרָאֵֽל׃ '

In [9]:
#TFOb(215297, BHSA).to_verses.pretty()

#### 1.3 List of occurrences from the DSS, 1QIsaa

In [10]:
verbs_dss = TFOb.all("scroll", DSS).filter(scroll="1Qisaa").to_words.filter_in(lex=motion_verbs)

In [22]:
#verbs_dss

#### Next step: get the verses from both databases

We cannot have an easy access to the clauses from the DSS database, for some reason?



In [14]:
def clean(g_cons):
    return g_cons.replace("_", " ").replace("׳", "").replace("'", "") 

In [15]:
verses_bhsa = []
verses_dss = []

for verb in chain(verbs_bhsa, verbs_dss):
    if verb.source.name == "BHSA":
        scroll = "MT"
        verse = verb.to_verses.to_words
        verses_bhsa.append(verse)
    else:
        scroll = verb.to_scrolls.scroll[0]
        verse = TFOb.section([verb.book[0], verb.chapter[0], verb.verse[0]], DSS, scroll)
        #verse = clean(str(verse))
        verses_dss.append(verse)

In [16]:
print(len(verses_bhsa))

571


In [17]:
print(len(verses_dss))

577


In [30]:
# Find the clause with the motion verb in the BHSA verses

clauses_bhsa = []

for verb in verbs_bhsa:
    mv_clause = verb.to_clauses
    clauses_bhsa.append(mv_clause)

# Find the complement in the mv_clause

cmpl_bhsa = []

for clause in clauses_bhsa:
    cmpl = clause.to_phrases.filter(function="Cmpl")
    cmpl_bhsa.append(cmpl)

In [31]:
#cmpl_bhsa

In [34]:
# Filter the clauses without a complement
cmpl_bhsa = [cmpl for cmpl in cmpl_bhsa if cmpl]
cmpl_bhsa      

[<phrase_1 "M NGD <JNJ">,
 <phrase_1 ">LJHM">,
 <phrase_1 "<LJK">,
 <phrase_1 ">L HR JHWH >L BJT >LHJ J<QB">,
 <phrase_1 "B >RXTJW">,
 <phrase_1 "M YJWN">,
 <phrase_1 "B >WR JHWH">,
 <phrase_1 "B YWR">,
 <phrase_1 "B M<RWT YRJM W B MXLWT <PR">,
 <phrase_1 "B NQRWT H YRJM W B S<PJ H SL<JM">,
 <phrase_1 "M JRWCLM W M JHWDH">,
 <phrase_2 "B MCPV <M ZQNJ <MW W FRJW">,
 <phrase_1 "B FDH">,
 <phrase_1 "M <L H MZBX">,
 <phrase_1 "JRWCLM">,
 <phrase_1 "B JHWDH">,
 <phrase_1 "<LJK W <L <MK W <L BJT >BJK">,
 <phrase_1 "M <L JHWDH">,
 <phrase_1 "CMH">,
 <phrase_1 "CMH">,
 <phrase_1 ">L H NBJ>H">,
 <phrase_1 "<LJHM">,
 <phrase_1 "<L KL >PJQJW">,
 <phrase_1 "<L KL GDWTJW">,
 <phrase_1 "B DRK H <M H ZH">,
 <phrase_1 "BH">,
 <phrase_1 "B J<QB">,
 <phrase_1 "B JFR>L">,
 <phrase_1 "M MRXQ">,
 <phrase_1 "<L MJ">,
 <phrase_1 "TXT HRWGJM">,
 <phrase_1 "B GWJ XNP">,
 <phrase_1 ">L >L GBWR">,
 <phrase_1 "BW">,
 <phrase_1 "M <L CKMK">,
 <phrase_1 "<L <JT">,
 <phrase_1 "B MGRWN">,
 <phrase_1 "M<BRH">,
 <phras

In [38]:
cmpl_bhsa[1].to_words.lex

['>L']

In [18]:
# Create the dataset

# Create a function that from a dss verb (TFOb object) returns a TFOb object containing the same verb from BHSA
# returns none is the verb is not the same

In [19]:
verbs_dss[0] # this is a TFOb object containing a verb

<word_1 "TB>W">

In [63]:
def find_bhsa_verb(verb_dss):
    """
    Checks if a verb occurring in DSS also occurs in BHSA (same book, chapter, verse, lexeme).
    Else, returns None.
    """
    
    #
    v_book = verb_dss.book[0]    
    v_chapter = verb_dss.chapter[0]
    v_verse = verb_dss.verse[0]

    #
    verse_bhsa = TFOb.section((v_book, v_chapter, v_verse), BHSA).to_words
    verb_bhsa = verse_bhsa.filter(lex=verb_dss.lex[0])
    
    # If repetition of verb in same verse: TODO
    if len(verb_bhsa) > 1:
        return # TODO
        scroll = verb_dss.to_scrolls.scroll[0]
        verse_dss = TFOb.section([v_book, v_chapter, v_verse], DSS, scroll)
        print("Verse BHSA:", verse_bhsa)
        print("Verse DSS:", verse_dss)
        
    if verb_bhsa:
        return verb_bhsa

    

#print(v_book, v_chapter, v_verse)
#print(verse_bhsa)

In [64]:
def find_dss_verse(verb_dss):
    """
    Returns the verse from the verb for a DSS occurrence.
    """
    # Extract the book, chapter, verse info form the verb
    book = verb_dss.book[0]    
    chapter = verb_dss.chapter[0]
    verse = verb_dss.verse[0]
    
    # Returns the verse
    scroll = verb_dss.to_scrolls.scroll[0]
    verse_dss = TFOb.section([book, chapter, verse], DSS, scroll)
    return verse_dss

In [86]:
# Test find_dss_verse function
print("BHSA verse: " + str(verbs_bhsa[1].to_verses)) # for comparison
print("DSS verse: " + str(find_dss_verse(verbs_dss[1]))) # function test

BHSA verse: L> TWSJPW HBJ> MNXT CW> QVRT TW<BH HJ> LJ XDC W CBT QR> MQR> L> >WKL >WN W <YRH
DSS verse: LW> TWSJPW L HBJ> MNXT CW> QVRT TW<BH HJ> LJ XWDC W CBT QR> MQR> LW> >WKL >WN W <YRTH


In [97]:
def is_lex_identical(verse_bhsa, verse_dss):
    """
    Checks if the verses (i.e. BHSA versus DSS) are identical on the lexeme level.
    """
    verse_without_spaces = [clean(lex) for lex in verse_bhsa.to_words.lex if lex] == [clean(lex) for lex in verse_dss.to_words.lex if lex]
    verse_with_spaces = [clean(lex) for lex in verse_bhsa.to_words.lex] == [clean(lex) for lex in verse_dss.to_words.lex]
    if verse_without_spaces != verse_with_spaces:
        print(verse_bhsa.to_words.lex)
        print(verse_dss.to_words.lex)
        print(verse_bhsa.text)
        print(verse_dss.text)
        print(verse_bhsa.to_words[-1].text)
        print(verse_dss.to_words[-1].text)
        

In [99]:
for i in range(0, len(verbs_bhsa)):
    print(f"\ni = {i}")
    print(is_lex_identical(verbs_bhsa[i].to_verses, find_dss_verse(verbs_dss[i]))) 


i = 0
None

i = 1
None

i = 2
None

i = 3
None

i = 4
None

i = 5
None

i = 6
None

i = 7
None

i = 8
['YJWN==/', 'B', 'MCPV/', 'PDH[', 'W', 'CWB[', 'B', 'YDQH/']
['YJWN==/', 'B', 'MCPV/', 'PDH[', 'W', 'CWB[', 'B', 'YDQH/', '']
צִיֹּ֖ון בְּמִשְׁפָּ֣ט תִּפָּדֶ֑ה וְשָׁבֶ֖יהָ בִּצְדָקָֽה׃ 
ציון במשפט תפדה ושבי׳ה בצדקה ׃ 
צְדָקָֽה׃ 
׃ 
None

i = 9
None

i = 10
None

i = 11
None

i = 12
None

i = 13
None

i = 14
['BJT/', 'J<QB/', 'HLK[', 'W', 'HLK[', 'B', '>WR/', 'JHWH/']
['BJT/', 'J<QB/', 'HLK[', 'W', 'HLK[', 'B', '>WR/', 'JHWH/', '']
בֵּ֖ית יַעֲקֹ֑ב לְכ֥וּ וְנֵלְכָ֖ה בְּאֹ֥ור יְהוָֽה׃ 
בית יעקוב לכו ונלכה באור יהוה ׃ 
יְהוָֽה׃ 
׃ 
None

i = 15
['BJT/', 'J<QB/', 'HLK[', 'W', 'HLK[', 'B', '>WR/', 'JHWH/']
['BJT/', 'J<QB/', 'HLK[', 'W', 'HLK[', 'B', '>WR/', 'JHWH/', '']
בֵּ֖ית יַעֲקֹ֑ב לְכ֥וּ וְנֵלְכָ֖ה בְּאֹ֥ור יְהוָֽה׃ 
בית יעקוב לכו ונלכה באור יהוה ׃ 
יְהוָֽה׃ 
׃ 
None

i = 16
None

i = 17
None

i = 18
None

i = 19
None

i = 20
None

i = 21
None

i = 22
['JHWH/', 'B', 'MCPV/', 'BW>[', '<

None

i = 280
None

i = 281
None

i = 282
None

i = 283
None

i = 284
None

i = 285
None

i = 286
None

i = 287
None

i = 288
None

i = 289
None

i = 290
None

i = 291
None

i = 292
None

i = 293
None

i = 294
None

i = 295
None

i = 296
None

i = 297
None

i = 298
None

i = 299
None

i = 300
None

i = 301
None

i = 302
None

i = 303
None

i = 304
None

i = 305
None

i = 306
None

i = 307
None

i = 308
None

i = 309
None

i = 310
None

i = 311
None

i = 312
None

i = 313
None

i = 314
None

i = 315
None

i = 316
None

i = 317
None

i = 318
None

i = 319
None

i = 320
None

i = 321
None

i = 322
None

i = 323
None

i = 324
None

i = 325
None

i = 326
None

i = 327
None

i = 328
None

i = 329
None

i = 330
None

i = 331
None

i = 332
None

i = 333
None

i = 334
None

i = 335
None

i = 336
None

i = 337
None

i = 338
None

i = 339
None

i = 340
None

i = 341
None

i = 342
None

i = 343
None

i = 344
None

i = 345
None

i = 346
None

i = 347
None

i = 348
None

i = 349
None

i = 350
None



In [88]:
# Print the results from is_lex_identical

n = 0

for i in range(0, len(verbs_bhsa)):
    if is_lex_identical(verbs_bhsa[i].to_verses, find_dss_verse(verbs_dss[i])):
        n += 1
print(n)


    #print(f"\ni = {i}")
    #print(is_lex_identical(verbs_bhsa[i].to_verses, find_dss_verse(verbs_dss[i])))  
    
# To see the lexemes, add the following lines inside the function is_lex_identical above:
#    print([clean(lex) for lex in verse_bhsa.to_words.lex if lex])
#    print([clean(lex) for lex in verse_dss.to_words.lex if lex])

47


In [76]:
# Create a dataset with the occurrences

def find_complements(verb):
    """Find the complement of a verb. If no match, returns None"""
    if verb.source.name == "BHSA":
        complements = verb.to_clauses.to_phrases.filter(function="Cmpl")
        return complements
    
    # if the verb is not BHSA, it's DSS
    verb_bhsa = find_bhsa_verb(verb)
        
    # Check if verses are identical
    if verb_bhsa and is_lex_identical(verb_bhsa.to_verses, verse):
        return find_complements(verb_bhsa)


items = [] # create an empty list to store all the information for each occ.

       
for verb in chain(verbs_bhsa, verbs_dss):
    
    # Add MT as "scroll" for the BHSA
    if verb.source.name == "BHSA":
        scroll = "MT"
        verse = verb.to_verses

    else:
        scroll = verb.to_scrolls.scroll[0]
        verse = TFOb.section([verb.book[0], verb.chapter[0], verb.verse[0]], DSS, scroll)
    
    complements = find_complements(verb)
    
    # If complements is None ==> there was no match between DSS and BHSA verses ==> find complement manually
    if complements is None:
        complements = [""]

    for complement in complements:
            
        # Collect information about the following variables:    
        item = {
            "verb_id": verb.ids[0], 
            "lex": verb.lex[0], 
            "scroll": scroll,
            "book": verb.book[0], 
            "chapter": verb.chapter[0], 
            "verse": verb.verse[0],
            "g_cons": clean(verb.g_cons[0]),
            "hebrew": verb.text,
            "g_cons_verse": clean(str(verse)),
            "stem": verb.vs[0],
            "tense": verb.vt[0],
            "subject": clean(str(verb.to_clauses.to_phrases.filter(function="Subj"))),
            "subj_heb": verb.to_clauses.to_phrases.filter(function="Subj").text,
            "complement": clean(str(complement)),
            "cmpl_heb" : verb.to_clauses.to_phrases.filter(function="Cmpl").text,
        }
        items.append(item)

In [101]:
df = pd.DataFrame(items).fillna("")
df.sort_values(["book", "chapter", "verse", "lex", "scroll"], ascending=[True, True, True, True, False], ignore_index=True, inplace=True) 

In [102]:
# Show the occurrences where DSS verse = BHSA verse
# df[(df.scroll != "MT") & (df.complement != "")]
df[(df.complement != "")]

,verb_id,lex,scroll,book,chapter,verse,g_cons,hebrew,g_cons_verse,stem,tense,subject,subj_heb,complement,cmpl_heb
2,212315,SWR[,MT,Isaiah,1,16,HSJRW,הָסִ֛ירוּ,RXYW HZKW HSJRW R< M<LLJKM M NGD <JNJ XDLW HR<,hif,impv,,,M NGD <JNJ,מִנֶּ֣גֶד עֵינָ֑י
5,212418,BW>[,MT,Isaiah,1,23,JBW>,יָבֹ֥וא,FRJK SWRRJM W XBRJ GNBJM KLW >HB CXD W RDP CLM...,qal,impf,RJB >LMNH,רִ֥יב אַלְמָנָ֖ה,>LJHM,אֲלֵיהֶֽם׃ פ
7,212437,CWB[,MT,Isaiah,1,25,>CJBH,אָשִׁ֤יבָה,W >CJBH JDJ <LJK W >YRP K BR SJGJK W >SJRH KL ...,hif,impf,,,<LJK,עָלַ֔יִךְ
16,215894,CWB[,MT,Isaiah,10,21,JCWB,יָשׁ֖וּב,C>R JCWB C>R J<QB >L >L GBWR,qal,impf,C>R C>R J<QB,שְׁאָ֥ר שְׁאָ֣ר יַעֲקֹ֑ב,>L >L GBWR,אֶל־אֵ֖ל גִּבֹּֽור׃
17,1898827,CWB[,1Qisaa,Isaiah,10,21,JCWB,ישוב,C>R JCWB C>R J<QWB >L >L GBWR,qal,impf,,,>L >L GBWR,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
743,214840,<LH[,MT,Isaiah,8,7,M<LH,מַעֲלֶ֨ה,W LKN HNH >DNJ M<LH <LJHM >T MJ H NHR H <YWMJM...,hif,ptca,>DNJ,אֲדֹנָי֩,<LJHM,עֲלֵיהֶ֜ם
744,214859,<LH[,MT,Isaiah,8,7,<LH,עָלָה֙,W LKN HNH >DNJ M<LH <LJHM >T MJ H NHR H <YWMJM...,qal,perf,,,<L KL >PJQJW,עַל־כָּל־אֲפִיקָ֔יו
747,214864,HLK[,MT,Isaiah,8,7,HLK,הָלַ֖ךְ,W LKN HNH >DNJ M<LH <LJHM >T MJ H NHR H <YWMJM...,qal,perf,,,<L KL GDWTJW,עַל־כָּל־גְּדֹותָֽיו׃
754,215292,CLX[,MT,Isaiah,9,7,CLX,שָׁלַ֥ח,DBR CLX >DNJ B J<QB W NPL B JFR>L,qal,perf,>DNJ,אֲדֹנָ֖י,B J<QB,בְּיַעֲקֹ֑ב


In [79]:
df

,verb_id,lex,scroll,book,chapter,verse,g_cons,hebrew,g_cons_verse,stem,tense,subject,subj_heb,complement,cmpl_heb
0,1895059,BW>[,1Qisaa,Isaiah,1,12,TB>W,תבאו,KJ> TB>W L R>WT PNJ MJ BQC ZW>T M JDKM L RMWS ...,qal,impf,,,,
1,1895075,BW>[,1Qisaa,Isaiah,1,13,HBJ>,הביא,LW> TWSJPW L HBJ> MNXT CW> QVRT TW<BH HJ> LJ X...,hifil,infc,,,,
2,212315,SWR[,MT,Isaiah,1,16,HSJRW,הָסִ֛ירוּ,RXYW HZKW HSJRW R< M<LLJKM M NGD <JNJ XDLW HR<,hif,impv,,,M NGD <JNJ,מִנֶּ֣גֶד עֵינָ֑י
3,1895129,SWR[,1Qisaa,Isaiah,1,16,HSJRW,הסירו,RXYW W HZKW W HSJRW RW< M<LLJKM M NGD <JNJ XDL...,hifil,impv,,,,
4,1895149,HLK[,1Qisaa,Isaiah,1,18,LKW,לכו,LKW N> W NWKXH JW>MR JHWH >M JHJW XV>JKM K CNJ...,qal,impv,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
753,1898435,CWB[,1Qisaa,Isaiah,9,20,CB,שב,W J>KL MNCH >T >PRJM W >PRJM >T MNCH JXDW HMH ...,qal,perf,,,,
754,215292,CLX[,MT,Isaiah,9,7,CLX,שָׁלַ֥ח,DBR CLX >DNJ B J<QB W NPL B JFR>L,qal,perf,>DNJ,אֲדֹנָ֖י,B J<QB,בְּיַעֲקֹ֑ב
755,1898190,CLX[,1Qisaa,Isaiah,9,7,CLX,שלח,DBR CLX JHWH B J<QWB W NPL B JFR>L,qal,perf,,,,
756,215297,NPL[,MT,Isaiah,9,7,NPL,נָפַ֖ל,DBR CLX >DNJ B J<QB W NPL B JFR>L,qal,perf,,,B JFR>L,בְּיִשְׂרָאֵֽל׃


In [25]:
df[df.scroll == "MT"]

,verb_id,lex,scroll,book,chapter,verse,g_cons,hebrew,g_cons_verse,stem,tense,subject,subj_heb,complement,cmpl_heb
2,212315,SWR[,MT,Isaiah,1,16,HSJRW,הָסִ֛ירוּ,RXYW HZKW HSJRW R< M<LLJKM M NGD <JNJ XDLW HR<,hif,impv,,,M NGD <JNJ,מִנֶּ֣גֶד עֵינָ֑י
5,212418,BW>[,MT,Isaiah,1,23,JBW>,יָבֹ֥וא,FRJK SWRRJM W XBRJ GNBJM KLW >HB CXD W RDP CLM...,qal,impf,RJB >LMNH,רִ֥יב אַלְמָנָ֖ה,>LJHM,אֲלֵיהֶֽם׃ פ
7,212437,CWB[,MT,Isaiah,1,25,>CJBH,אָשִׁ֤יבָה,W >CJBH JDJ <LJK W >YRP K BR SJGJK W >SJRH KL ...,hif,impf,,,<LJK,עָלַ֔יִךְ
16,215894,CWB[,MT,Isaiah,10,21,JCWB,יָשׁ֖וּב,C>R JCWB C>R J<QB >L >L GBWR,qal,impf,C>R C>R J<QB,שְׁאָ֥ר שְׁאָ֣ר יַעֲקֹ֑ב,>L >L GBWR,אֶל־אֵ֖ל גִּבֹּֽור׃
18,215910,CWB[,MT,Isaiah,10,22,JCWB,יָשׁ֣וּב,KJ >M JHJH <MK JFR>L K XWL H JM C>R JCWB BW KL...,qal,impf,C>R,שְׁאָ֖ר,BW,בֹּ֑ו
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
801,214840,<LH[,MT,Isaiah,8,7,M<LH,מַעֲלֶ֨ה,W LKN HNH >DNJ M<LH <LJHM >T MJ H NHR H <YWMJM...,hif,ptca,>DNJ,אֲדֹנָי֩,<LJHM,עֲלֵיהֶ֜ם
802,214859,<LH[,MT,Isaiah,8,7,<LH,עָלָה֙,W LKN HNH >DNJ M<LH <LJHM >T MJ H NHR H <YWMJM...,qal,perf,,,<L KL >PJQJW,עַל־כָּל־אֲפִיקָ֔יו
805,214864,HLK[,MT,Isaiah,8,7,HLK,הָלַ֖ךְ,W LKN HNH >DNJ M<LH <LJHM >T MJ H NHR H <YWMJM...,qal,perf,,,<L KL GDWTJW,עַל־כָּל־גְּדֹותָֽיו׃
813,215292,CLX[,MT,Isaiah,9,7,CLX,שָׁלַ֥ח,DBR CLX >DNJ B J<QB W NPL B JFR>L,qal,perf,>DNJ,אֲדֹנָ֖י,B J<QB,בְּיַעֲקֹ֑ב


In [26]:
df[df.scroll != "MT"]

,verb_id,lex,scroll,book,chapter,verse,g_cons,hebrew,g_cons_verse,stem,tense,subject,subj_heb,complement,cmpl_heb
0,1895059,BW>[,1Qisaa,Isaiah,1,12,TB>W,תבאו,KJ> TB>W L R>WT PNJ MJ BQC ZW>T M JDKM L RMWS ...,qal,impf,,,,
1,1895075,BW>[,1Qisaa,Isaiah,1,13,HBJ>,הביא,LW> TWSJPW L HBJ> MNXT CW> QVRT TW<BH HJ> LJ X...,hifil,infc,,,,
3,1895129,SWR[,1Qisaa,Isaiah,1,16,HSJRW,הסירו,RXYW W HZKW W HSJRW RW< M<LLJKM M NGD <JNJ XDL...,hifil,impv,,,,
4,1895149,HLK[,1Qisaa,Isaiah,1,18,LKW,לכו,LKW N> W NWKXH JW>MR JHWH >M JHJW XV>JKM K CNJ...,qal,impv,,,,
6,1895234,BW>[,1Qisaa,Isaiah,1,23,JBW>,יבוא,FRJKJ SWRRJM W XBRJ GNBJM KWLM >WHBJ CWXD RWDP...,qal,impf,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
810,1898270,CWB[,1Qisaa,Isaiah,9,12,CB,שב,W H <M LW> CB <L H MKHW W >T JHWH YB>WT LW> DRCW,qal,perf,,,,
811,1898352,CWB[,1Qisaa,Isaiah,9,16,CB,שב,<L KN <L BXWRJW LW> JXMWL >DWNJ W >T JTWMJW W ...,qal,perf,,,,
812,1898435,CWB[,1Qisaa,Isaiah,9,20,CB,שב,W J>KL MNCH >T >PRJM W >PRJM >T MNCH JXDW HMH ...,qal,perf,,,,
814,1898190,CLX[,1Qisaa,Isaiah,9,7,CLX,שלח,DBR CLX JHWH B J<QWB W NPL B JFR>L,qal,perf,,,,


In [ ]:
# Compare the lexemes and their order
# clean the DSS verses by Id to be able to retrieve the good complements (and not copy them from BHSA)